In [2]:
pip install datasets transformers torch unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 137.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393

In [3]:
from datasets import load_dataset

dataset = load_dataset("ai4privacy/pii-masking-400k")
train_data = dataset["train"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

1en.jsonl:   0%|          | 0.00/84.8M [00:00<?, ?B/s]

de.jsonl:   0%|          | 0.00/82.7M [00:00<?, ?B/s]

es.jsonl:   0%|          | 0.00/42.5M [00:00<?, ?B/s]

...

fr.jsonl:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

it.jsonl:   0%|          | 0.00/19.8M [00:00<?, ?B/s]

nl.jsonl:   0%|          | 0.00/9.67M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/325517 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/81379 [00:00<?, ? examples/s]

In [4]:
print(train_data.column_names)

['source_text', 'locale', 'language', 'split', 'privacy_mask', 'uid', 'masked_text', 'mbert_tokens', 'mbert_token_classes']


In [5]:
def format_data(example):
    return {
        "input": example["source_text"],
        "output": example["masked_text"]
    }

formatted_data = train_data.map(format_data)

Map:   0%|          | 0/325517 [00:00<?, ? examples/s]

In [6]:
def format_chat(example):
    messages = [
        {"role": "system", "content": "You are a privacy assistant. Redact all personally identifiable information (PII) from the following text."},
        {"role": "user", "content": example["source_text"]},
        {"role": "assistant", "content": example["masked_text"]}
    ]
    return {"messages": messages}

formatted_chat_data = train_data.map(format_chat)

Map:   0%|          | 0/325517 [00:00<?, ? examples/s]

In [7]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.version.cuda)  # Should match your CUDA version

True
12.6


In [8]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct", # "unsloth/Phi-3-mini-4k-instruct loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Mistral patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [9]:
from unsloth.chat_templates import get_chat_template

In [10]:
def tokenize_function(example):
    # tokenizer.apply_chat_template returns a tensor if return_tensors="pt"
    # We need to return a dict with a key like "input_ids"
    input_ids = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=True,
        add_generation_prompt=False,
        return_tensors="pt"
    )
    # Convert to list so it can be stored in the dataset
    return {"input_ids": input_ids[0].tolist()}

dataset_tokenized = formatted_chat_data.map(tokenize_function)

Map:   0%|          | 0/325517 [00:00<?, ? examples/s]

In [16]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['source_text', 'locale', 'language', 'split', 'privacy_mask', 'uid', 'masked_text', 'mbert_tokens', 'mbert_token_classes'],
        num_rows: 325517
    })
    validation: Dataset({
        features: ['source_text', 'locale', 'language', 'split', 'privacy_mask', 'uid', 'masked_text', 'mbert_tokens', 'mbert_token_classes'],
        num_rows: 81379
    })
})


In [ ]:
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"CUDA: {torch.version.cuda}")
print(f"BF16 support: {torch.cuda.is_bf16_supported()}")

In [19]:
import os
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Environment fSetup for T4 GPUs
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Helps with debugging

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenization with dynamic padding
dataset_tokenized = dataset.map(
    lambda x: tokenizer(
        x["masked_text"],
        truncation=True,
        max_length=1024,
        return_tensors=None,
    ),
    batched=True,
    remove_columns=dataset["train"].column_names
)

# LoRA configuration remains the same
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["qkv_proj", "o_proj", "down_proj", "up_proj", "gate_proj"],
    lora_alpha=16,
    lora_dropout=0,
)

# Updated training arguments for T4 compatibility
training_args = TrainingArguments(
    output_dir="./results",
    report_to="none",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    fp16=True,  # Use fp16 instead of bf16 for T4 compatibility
    learning_rate=2e-5,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=1000,
    optim="adamw_torch",
    gradient_checkpointing=True,
    torch_compile=False,  # Disable torch.compile for stability
)

# Data collator remains unchanged
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_tokenized["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    preprocess_logits_for_metrics=lambda logits, labels: logits
)

# Start training
trainer.train()

Map:   0%|          | 0/325517 [00:00<?, ? examples/s]

Unsloth: Already have LoRA adapters! We shall skip this step.
<ipython-input-19-26857f949d3d>:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 325,517 | Num Epochs = 1 | Total steps = 81,380
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 20,447,232/4,000,000,000 (0.51% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
100,3.301500
200,2.900700
300,2.570300


KeyboardInterrupt: 

In [ ]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

In [ ]:
FastLanguageModel.for_inference(model)
messages = [
    {"role": "system", "content": "You are a privacy assistant. Redact all personally identifiable information (PII) from the following text."},
    {"role": "user", "content": "My name is Abhishek Sharma and my email is something.something@example.com."}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")
outputs = model.generate(inputs, max_new_tokens=128)
print(tokenizer.batch_decode(outputs)[0])